# User recognition

Для начала необходимо определить ключ, по которому мы сможем отделять запросы одного пользователя от другого. На выбор в данных есть поля:
* fingerprint
* src_ip
* ua_header
* cookie

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sys

pd.set_option('display.max_colwidth', 2000)

## 1. Data loading and preprocessing

In [2]:
dtype = {
    'time': 'str',
    'fingerprint': 'uint',
    'src_ip': 'uint',
    'ua_header': 'str',
    'cookie': 'str',
    'request_uri': 'str',
}

In [3]:
data = pd.read_csv('data.csv', parse_dates=['time'], dtype=dtype)
data.head()

,time,fingerprint,src_ip,ua_header,cookie,request_uri
0,2022-02-07 11:40:25.020,10079584030241655126,54597,got (https://github.com/sindresorhus/got),NaN,/goods/324570/gz0473/
1,2022-02-07 11:40:25.026,10079584030241655126,54597,got (https://github.com/sindresorhus/got),NaN,/goods/338875/dh3718-105/
2,2022-02-07 11:40:25.046,9446535646632202739,54651,"Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X)AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1",4ZsUJ6d58xnrLFmPBaQY,/getproductsize/339316/
3,2022-02-07 11:40:25.047,9446535646632202739,54651,"Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X)AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1",4ZsUJ6d58xnrLFmPBaQY,/getproductsize/342531/
4,2022-02-07 11:40:25.052,9446535646632202739,54651,"Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X)AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1",4ZsUJ6d58xnrLFmPBaQY,/getproductsize/338875/


In [9]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5461175 entries, 0 to 5461174
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   time         datetime64[ns]
 1   fingerprint  uint64        
 2   src_ip       uint64        
 3   ua_header    object        
 4   cookie       object        
 5   request_uri  object        
dtypes: datetime64[ns](1), object(3), uint64(2)
memory usage: 1.7 GB


In [10]:
data.isna().sum()

time                0
fingerprint         0
src_ip              0
ua_header         579
cookie         913575
request_uri         0
dtype: int64

In [11]:
data = data.fillna('empty')

In [17]:
data['uri_cutted'] = data.request_uri.apply(lambda x: x[:50])

## 2. EDA

Мы не знаем, сколько реальных людей посещают сайт в течение суток, но количество уникальных ключей каждого поля сильно разнится.

In [12]:
data[['fingerprint', 'src_ip', 'ua_header', 'cookie']].agg('nunique')

fingerprint     31825
src_ip          67638
ua_header       12017
cookie         136698
dtype: int64

fingerprint представляет из себя хэшированные параметры шифрования и некоторые http-заголовки, т.е. это отпечаток устройства, с которого совершается запрос. Устройства могут совпадать, а т.к. fingerprint не содержит сетевой информации (только информацию об автономной системе), то для идентификации в чистом виде он не подойдет. А значит можно использовать несколько полей для идентификации. Чем больше полей мы возmмем, тем выше будет уникальность, что тоже нехорошо.

Далее детальней посмотрим на другие параметры.

__cookie:__

Куки могли бы быть отличным идентификатором, т.к. это уникальный номер, который дается на стороне сервера и он сам по себе необходим для идентификации пользователя на сайте. Но очень много пустых кук в данных. По-умолчанию при первом запросе куки могут быть пустыми. Куки могут не выдаваться при автоматическом трафике.

In [13]:
data[data.cookie == 'empty'].shape[0] / data.shape[0]

0.16728542850210806

In [14]:
fp_group = data.groupby('fingerprint').nunique()
fp_group.sort_values('cookie')[['cookie']]

,cookie
fingerprint,
246859976946643,1
11025727900325393915,1
11025151725081528285,1
11024184968607764450,1
11023061947250533930,1
...,...
17256714284921906028,6495
3198524912015451322,10737
17521262463243347016,12711


Много фингерпринтов, для которых встречается очень большое количество уникальных кук. Ниже пара примеров.

Это нормальная ситуация, когда мы видим реальных пользователей и пара fingerprint + cookie хорошо справляется. При этом разные cookie соответствуют разным ip-адресам, и в данном случае мы могли бы даже использовать пару fingerprint + src_ip, чтобы не учитывать пустую куку на первом запросе.

In [18]:
data[data.fingerprint == 4114659945210122468].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted
13,2022-02-07 11:40:25.139,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/new/?mfp=31-kategoriya%5B%D0%9A%D1%80%D0%BE%D1%81
45,2022-02-07 11:40:25.384,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/342434/gw9525/
62,2022-02-07 11:40:25.557,4114659945210122468,19553,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,BCbxe29fgXjOFjkLwA6t,/goods/299420/gz3079/
67,2022-02-07 11:40:25.608,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,empty,/new/?mfp=31-kategoriya%5B%D0%9A%D1%80%D0%BE%D1%81
86,2022-02-07 11:40:26.034,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,xYVNFJO6iTcGV7O7XaZt,/goods/338875/dh3718-105/
89,2022-02-07 11:40:26.090,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/335987/gy4109/
92,2022-02-07 11:40:26.141,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/299420/gz3079/
120,2022-02-07 11:40:26.418,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,xYVNFJO6iTcGV7O7XaZt,/goods/327166/dd7099-100/
123,2022-02-07 11:40:26.447,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,xYVNFJO6iTcGV7O7XaZt,/goods/314556/dh3718-100/
126,2022-02-07 11:40:26.487,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/319689/gz0724/


А вот уже пример, где на самом деле ни одна пара не сработает. Мы видим, что это боты скорее всего из одной сети, куки выдаются как будто рандомно и могут быть пустыми даже для устройства, которое уже совершало запрос (строка 766 и 2635). Перебираются и адреса и ua_header.

In [19]:
data[data.fingerprint == 17521262463243347016].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted
766,2022-02-07 11:40:34.776,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",lrpJgqyWEW9JqPRPwQfL,/goods/337371/dj1291-320/
769,2022-02-07 11:40:34.812,17521262463243347016,14028,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),empty,/goods/45667/muzhskie-krossovki-new-balance-hl754b
1048,2022-02-07 11:40:37.932,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",empty,/goods/331965/na4eu9041/
1772,2022-02-07 11:40:46.801,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",ZBNnB914VWHuNyOzxBL3,/goods/10397/kurtka-anorak-zhenskaya-napapijri-ski
2485,2022-02-07 11:40:55.596,17521262463243347016,14029,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),empty,/goods/298500/ml574shp/
2596,2022-02-07 11:40:56.701,17521262463243347016,14027,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",13AmCl07Aqn0zO1tgjuo,/nike/
2635,2022-02-07 11:40:57.176,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",empty,/goods/315384/va5dxz9fg/
2798,2022-02-07 11:40:59.216,17521262463243347016,14027,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",2014FnqBz4zOuqPHLIdt,/goods/300474/m5740tb/
3152,2022-02-07 11:41:04.010,17521262463243347016,14029,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),empty,/goods/315136/va5fj7dz9/
3721,2022-02-07 11:41:11.800,17521262463243347016,14027,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",FPcZuKVsjRSEN8fUHLMm,/goods/97544/muzhskoe-polo-stone-island-patch-prog


__src_ip__

Ниже я покажу очевидный пример, почему так же пара fingerprint + src_ip не подойдет.

In [20]:
fp_group.sort_values('src_ip')[['src_ip']]

,src_ip
fingerprint,
246859976946643,1
11221527933842028937,1
11220229799712395838,1
11219387462976806329,1
11219222025557612999,1
...,...
16877047899345133502,1401
15166714124895094622,1490
15286119680973688619,1783


Здесь мы видим явного бота, который подменяет ip-адреса на каждом запросе. Но при этом все куки пустые, что служит хорошим триггером.

In [21]:
data[data.fingerprint == 14245688443241137919].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted
2531,2022-02-07 11:40:55.999,14245688443241137919,30216,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/339331/
2547,2022-02-07 11:40:56.149,14245688443241137919,61820,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/334222/
2557,2022-02-07 11:40:56.293,14245688443241137919,6551,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/327647/
3858,2022-02-07 11:41:13.290,14245688443241137919,19548,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/326325/
3867,2022-02-07 11:41:13.382,14245688443241137919,48607,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/327585/
3875,2022-02-07 11:41:13.494,14245688443241137919,48307,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/336511/
3882,2022-02-07 11:41:13.593,14245688443241137919,27848,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/332414/
3883,2022-02-07 11:41:13.615,14245688443241137919,51639,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/324241/
4034,2022-02-07 11:41:15.532,14245688443241137919,32878,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/344337/
4065,2022-02-07 11:41:15.720,14245688443241137919,63,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/342539/


## 3. User recognition

#### 3.1. Experiments

Использовать в паре user_agent для идентификации так же не стоит, так как это первое, что подменить проще всего. Хотя пример выше показывает, что не все так делают.

Поэтому я предлагаю свой метод идентификации: 
* по-умолчанию мы используем главный ключ - пару fingerprint + src_ip.
* в кэше храним дополнительный ключ - пару fingerprint + ua_header.
* если мы видим пустые куки и дополнительный ключ в кэше, то мы назначаем в качестве идентификатора ключ из кэша. Это поможет определить ботов, меняющих адреса.
* если мы видим пустые куки и дополнительного ключа в кэше нет, то кладем ключи в кэш.
* если же мы видим не пустуые куки, но дополнительный ключ есть в кэше, мы его берем из кэша и удаляем. Это гарантирует нормальную идентификацию обычного пользователя, который получил куки на первом запросе.

Далее рассмотрю работу алгоритма на 3 примерах выше.

In [22]:
slice_3_users = data[data.fingerprint == 4114659945210122468].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6]]
slice_1_bot = data[data.fingerprint == 14245688443241137919].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6]]
df_slice = data[data.fingerprint == 17521262463243347016].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6]]

In [23]:
def get_user_ids(df):
    uids = []
    fp_ua_cache = {}
    for row in tqdm(df.itertuples(), total=df.shape[0]):
        uid = hash(tuple((row.fingerprint, row.src_ip))) + sys.maxsize + 1
        fpid = hash(tuple((row.fingerprint, row.ua_header))) + sys.maxsize + 1
        if row.cookie != 'empty':
            if fpid in fp_ua_cache:
                uid = fp_ua_cache.pop(fpid)
        else:
            if fpid in fp_ua_cache:
                uid = fp_ua_cache[fpid]
            else:
                fp_ua_cache[fpid] = uid
        uids.append(uid)
    return uids

In [24]:
slice_3_users['uid'] = get_user_ids(slice_3_users)
slice_3_users

  0%|          | 0/10 [00:00<?, ?it/s]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,uid
13,2022-02-07 11:40:25.139,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/new/?mfp=31-kategoriya%5B%D0%9A%D1%80%D0%BE%D1%81,10724200188977751206
45,2022-02-07 11:40:25.384,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/342434/gw9525/,10724200188977751206
62,2022-02-07 11:40:25.557,4114659945210122468,19553,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,BCbxe29fgXjOFjkLwA6t,/goods/299420/gz3079/,276531018559180099
67,2022-02-07 11:40:25.608,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,empty,/new/?mfp=31-kategoriya%5B%D0%9A%D1%80%D0%BE%D1%81,10508021874908110880
86,2022-02-07 11:40:26.034,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,xYVNFJO6iTcGV7O7XaZt,/goods/338875/dh3718-105/,10508021874908110880
89,2022-02-07 11:40:26.090,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/335987/gy4109/,10724200188977751206
92,2022-02-07 11:40:26.141,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/299420/gz3079/,10724200188977751206
120,2022-02-07 11:40:26.418,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,xYVNFJO6iTcGV7O7XaZt,/goods/327166/dd7099-100/,10508021874908110880
123,2022-02-07 11:40:26.447,4114659945210122468,19557,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,xYVNFJO6iTcGV7O7XaZt,/goods/314556/dh3718-100/,10508021874908110880
126,2022-02-07 11:40:26.487,4114659945210122468,19572,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0,CRjjqfBtZI2ZxoC271fn,/goods/319689/gz0724/,10724200188977751206


In [25]:
slice_1_bot['uid'] = get_user_ids(slice_1_bot)
slice_1_bot

  0%|          | 0/10 [00:00<?, ?it/s]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,uid
2531,2022-02-07 11:40:55.999,14245688443241137919,30216,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/339331/,2998136084459279765
2547,2022-02-07 11:40:56.149,14245688443241137919,61820,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/334222/,2998136084459279765
2557,2022-02-07 11:40:56.293,14245688443241137919,6551,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/327647/,2998136084459279765
3858,2022-02-07 11:41:13.290,14245688443241137919,19548,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/326325/,2998136084459279765
3867,2022-02-07 11:41:13.382,14245688443241137919,48607,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/327585/,2998136084459279765
3875,2022-02-07 11:41:13.494,14245688443241137919,48307,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/336511/,2998136084459279765
3882,2022-02-07 11:41:13.593,14245688443241137919,27848,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/332414/,2998136084459279765
3883,2022-02-07 11:41:13.615,14245688443241137919,51639,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/324241/,2998136084459279765
4034,2022-02-07 11:41:15.532,14245688443241137919,32878,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/344337/,2998136084459279765
4065,2022-02-07 11:41:15.720,14245688443241137919,63,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/342539/,2998136084459279765


In [26]:
df_slice['uid'] = get_user_ids(df_slice)
df_slice

  0%|          | 0/10 [00:00<?, ?it/s]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,uid
766,2022-02-07 11:40:34.776,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",lrpJgqyWEW9JqPRPwQfL,/goods/337371/dj1291-320/,7378769679882211089
769,2022-02-07 11:40:34.812,17521262463243347016,14028,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),empty,/goods/45667/muzhskie-krossovki-new-balance-hl754b,7378769679882211089
1048,2022-02-07 11:40:37.932,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",empty,/goods/331965/na4eu9041/,7378769679882211089
1772,2022-02-07 11:40:46.801,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",ZBNnB914VWHuNyOzxBL3,/goods/10397/kurtka-anorak-zhenskaya-napapijri-ski,7378769679882211089
2485,2022-02-07 11:40:55.596,17521262463243347016,14029,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),empty,/goods/298500/ml574shp/,7378769679882211089
2596,2022-02-07 11:40:56.701,17521262463243347016,14027,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",13AmCl07Aqn0zO1tgjuo,/nike/,12271843708854907455
2635,2022-02-07 11:40:57.176,17521262463243347016,14028,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",empty,/goods/315384/va5dxz9fg/,7378769679882211089
2798,2022-02-07 11:40:59.216,17521262463243347016,14027,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",2014FnqBz4zOuqPHLIdt,/goods/300474/m5740tb/,7378769679882211089
3152,2022-02-07 11:41:04.010,17521262463243347016,14029,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),empty,/goods/315136/va5fj7dz9/,7378769679882211089
3721,2022-02-07 11:41:11.800,17521262463243347016,14027,"Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",FPcZuKVsjRSEN8fUHLMm,/goods/97544/muzhskoe-polo-stone-island-patch-prog,12271843708854907455


Во всех случаях алгоритм справился хорошо. В последнем случае получилось 2 пользователя, хотя мы понимаем, что источник один.

#### 3.2. UID making for all data

In [27]:
data = data.sort_values('time').reset_index(drop=True)
data['user_id'] = get_user_ids(data)

  0%|          | 0/5461175 [00:00<?, ?it/s]

## 4. Results

In [28]:
user_groups = data.groupby('user_id').nunique()

Получилось 133 тысячи уникальных пользователей. Далее посмотрим, как алгоритм сработал на примере пользователей с наиболее уникальными полями.

In [29]:
user_groups.sort_values('src_ip')[['src_ip']]

,src_ip
user_id,
141794952076039,1
12229715140777011424,1
12229409071612906219,1
12229138800768931454,1
12228856536967170642,1
...,...
1440682179867697865,430
4424099593356133400,553
15157763850920392037,573


Перебор ip-адресов - хорошо.

In [30]:
data[data.user_id == 2998136084459279765].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6, 7]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,user_id
2531,2022-02-07 11:40:55.999,14245688443241137919,30216,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/339331/,2998136084459279765
2547,2022-02-07 11:40:56.149,14245688443241137919,61820,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/334222/,2998136084459279765
2557,2022-02-07 11:40:56.293,14245688443241137919,6551,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/327647/,2998136084459279765
3858,2022-02-07 11:41:13.290,14245688443241137919,19548,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/326325/,2998136084459279765
3867,2022-02-07 11:41:13.382,14245688443241137919,48607,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/327585/,2998136084459279765
3875,2022-02-07 11:41:13.494,14245688443241137919,48307,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/336511/,2998136084459279765
3882,2022-02-07 11:41:13.593,14245688443241137919,27848,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/332414/,2998136084459279765
3883,2022-02-07 11:41:13.615,14245688443241137919,51639,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/324241/,2998136084459279765
4034,2022-02-07 11:41:15.532,14245688443241137919,32878,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/344337/,2998136084459279765
4065,2022-02-07 11:41:15.720,14245688443241137919,63,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",empty,/getproductsize/342539/,2998136084459279765


In [31]:
user_groups.sort_values('cookie')[['cookie']]

,cookie
user_id,
9221839682022329965,1
11642190754847451010,1
11642185557923309736,1
11642040653990720457,1
11641865794808370260,1
...,...
13206274927096171458,8576
10025968614691915469,8578
14355576028179037656,8582


Мы видим одну сеть, один fingerprint, один браузер и очень похожие периодичные запросы, но разные куки. Вероятнее всего, это один пользователь - бот.

In [32]:
data[data.user_id == 7295268216093594189].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6, 7]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,user_id
408,2022-02-07 11:40:30.169,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,fsE2MEFlHJwY2fkx2tcC,/getproductsize/339316/,7295268216093594189
444,2022-02-07 11:40:30.350,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,fsE2MEFlHJwY2fkx2tcC,/getproductsize/344422/,7295268216093594189
815,2022-02-07 11:40:35.338,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,QuOV64U7KwQcj14tSfu0,/getproductsize/335987/,7295268216093594189
816,2022-02-07 11:40:35.340,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,QuOV64U7KwQcj14tSfu0,/getproductsize/336511/,7295268216093594189
1241,2022-02-07 11:40:40.665,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,1ZH8eV7U4fWqtHKEHEkU,/getproductsize/327166/,7295268216093594189
1250,2022-02-07 11:40:40.704,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,1ZH8eV7U4fWqtHKEHEkU,/getproductsize/324565/,7295268216093594189
2225,2022-02-07 11:40:52.771,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,llNSIej5HhKlzgH5yOgV,/new/?mfp=31-kategoriya%5B%D0%9A%D1%80%D0%BE%D1%81,7295268216093594189
2419,2022-02-07 11:40:54.695,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,llNSIej5HhKlzgH5yOgV,/getproductsize/342434/,7295268216093594189
2826,2022-02-07 11:40:59.562,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,zJYtInlnWTQG7hiNZ3mF,/getproductsize/336501/,7295268216093594189
2827,2022-02-07 11:40:59.564,2029279384994103086,36413,Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0,zJYtInlnWTQG7hiNZ3mF,/getproductsize/336506/,7295268216093594189


In [33]:
user_groups.sort_values('ua_header')[['ua_header']]

,ua_header
user_id,
141794952076039,1
12176526248337312268,1
12176210872377834882,1
12176156666293828725,1
12176105506307865483,1
...,...
8548697001057556512,30
8704836898415406754,235
17508519724759123706,249


Явные переборы ua_header - хорошо.

In [34]:
data[data.user_id == 4047652270815651211].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6, 7]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,user_id
122,2022-02-07 11:40:26.434,7993457820035721372,5922,"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2941.53 Safari/537.36",J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
229,2022-02-07 11:40:27.872,7993457820035721372,5922,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:48.0) Gecko/20100101 Firefox/48.0,J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
3146,2022-02-07 11:41:03.930,7993457820035721372,5922,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2797.18 Safari/537.36",J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
3352,2022-02-07 11:41:06.642,7993457820035721372,5922,"Mozilla/5.0 (X11; Ubuntu; Linux i686 on x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2680.54 Safari/537.36",J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
3559,2022-02-07 11:41:09.590,7993457820035721372,5922,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2816.71 Safari/537.36",J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
4967,2022-02-07 11:41:28.075,7993457820035721372,5922,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2934.21 Safari/537.36",J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
6679,2022-02-07 11:41:50.800,7993457820035721372,5922,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2850.54 Safari/537.36",J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
6829,2022-02-07 11:41:52.752,7993457820035721372,5922,Mozilla/5.0 (X11; Linux x86_64; rv:49.0) Gecko/20100101 Firefox/49.0,J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
7398,2022-02-07 11:42:01.068,7993457820035721372,5922,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:48.0) Gecko/20100101 Firefox/48.0,J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211
10137,2022-02-07 11:42:36.536,7993457820035721372,5922,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:45.0) Gecko/20100101 Firefox/45.0,J8qdmfVqHNw729Uqw15M,/xhr/login/activationcode/,4047652270815651211


In [35]:
data.groupby(['src_ip', 'cookie']).nunique().sort_values('fingerprint')[['fingerprint']]

fingerprint
src_ip cookie                           
35128  RSHFic24Bd7gpwjCPkoS            1
36523  n7p4hi3vJ8u3jfqoG261            1
       n7Py37wrmCDQoAMa0AaL            1
       n7BmfZTG0gfDUC3ZAfcu            1
       n6cf6Ty8wuCOOthHbRqv            1
...                                  ...
61554  AEv5OzMEBukoejn1RJJg           14
36760  empty                          15
17106  empty                          15
6313   t2HNtwJ3HBWNswB7cuuZ           20
21489  EmouT3b6rQtbSFX8xV2L           22

[561608 rows x 1 columns]

Опять странная ситуация - одна сеть, одни и те же куки и браузер, но разные фингерпринты. Запросы похожи на обычные пользовательские запросы с браузера (загрузка статики и скриптов). Но разные фингерпринты. С учетом того, что таких случаев на порядки меньше, чем переборов других полей, сочтем, что можно пренебречь такими спорными ситуациями.

In [36]:
data[data.cookie == 'EmouT3b6rQtbSFX8xV2L'].sort_values('time').iloc[:10, [0, 1, 2, 3, 4, 6, 7]]

,time,fingerprint,src_ip,ua_header,cookie,uri_cutted,user_id
2499727,2022-02-07 20:47:11.153,14080841322074431192,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/view/theme/example2019/js/libs/widjet.js,16366869651969497480
2499773,2022-02-07 20:47:11.811,5902107132065066494,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/xhr/cart/,848996465577121059
2499775,2022-02-07 20:47:11.823,15780113147905621085,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/xhr/productsliders/?slider_id=14,15854636468651557601
2499777,2022-02-07 20:47:11.834,7891483658658700140,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/xhr/productsliders/?slider_id=18,14897049102183508248
2500474,2022-02-07 20:47:22.780,14917351027282694254,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/muzhskoe/verhnyaya-odezhda/kurtki/,5359739514297969552
2500521,2022-02-07 20:47:23.265,9364456603781239295,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/xhr/cart/,8968930874577037642
2500736,2022-02-07 20:47:26.337,11619217537108571595,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/favicon.ico?v=07022022181900,15102472115135435836
2500833,2022-02-07 20:47:27.671,17972872883797205511,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/muzhskoe/odezhda/tolstovki/,12639912636370273824
2500858,2022-02-07 20:47:28.183,14917351027282694254,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/xhr/cart/,5359739514297969552
2500878,2022-02-07 20:47:28.405,3012939365011799917,21489,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",EmouT3b6rQtbSFX8xV2L,/view/theme/example2019/img/icons/preload/preload.,709441107241001359


In [37]:
data.loc[:, ['time', 'user_id', 'request_uri']].to_csv('data_with_uids.csv', index=False)